<a href="https://colab.research.google.com/github/woodstone10/data-collection/blob/main/web_crawling_and_data_framing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# web crawling and data framing with Pandas



In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def crawl(url):
    try:
        rData = requests.get(url)  # web crawling
        print(rData)  # expect Response [200]
        return rData.content
    except:
        print("<Response Error>")
        return 0

def parse(str):
    bsData = BeautifulSoup(str, "html.parser")  
    name = bsData.find("h1", {"class": "KY7mAb"})  # < h1 class ="KY7mAb" > Company name < / h1 >
    price = bsData.find("div", {"class": "YMlKec fxKbKc"})  # <div class="YMlKec fxKbKc">Price $xx.xx</div>
    return {"name": name.text, "price": price.text}

In [ ]:
url = ["https://www.google.com/finance/quote/VZ:NYSE",  # Verizon
        "https://www.google.com/finance/quote/QCOM:NASDAQ",  # Qualcomm
        "https://www.error", # wrong page for error handling test purpose
        "https://www.google.com/finance/quote/AAPL:NASDAQ",
        "https://www.google.com/finance/quote/AMZN:NASDAQ",
        "https://www.google.com/finance/quote/TMUS:NASDAQ",
        "https://www.google.com/finance/quote/TSLA:NASDAQ",
        ]

data = []
for i in range(len(url)):
    str = crawl(url[i])
    if str == 0:
        continue
    txt = parse(str)
    name = txt.get("name")
    price = txt.get("price")
    print(name, ":", price)
    data.append([name, float(price.split('$')[1].replace(',',''))]) #fix convert string to float due to comma

<Response [200]>
Verizon Communications Inc. : $58.39
<Response [200]>
QUALCOMM, Inc. : $135.25
<Response Error>
<Response [200]>
Apple Inc : $134.94
<Response [200]>
Amazon.com, Inc. : $3,372.01
<Response [200]>
T-Mobile Us Inc : $131.99
<Response [200]>
Tesla Inc : $714.63


## Data framing with Pandas

In [ ]:
import pandas as pd
df = pd.DataFrame(data, columns = ['Company Name','Stock Price']) #Pandas data frame
df

,Company Name,Stock Price
0,Verizon Communications Inc.,58.39
1,"QUALCOMM, Inc.",135.25
2,Apple Inc,134.94
3,"Amazon.com, Inc.",3372.01
4,T-Mobile Us Inc,131.99
5,Tesla Inc,714.63


In [ ]:
df.to_excel("output.xlsx", sheet_name='sheet1') #Save to Excel

### extract

In [ ]:
def filter_keyword(data, keyword):
  result = data[data['Company Name'].str.contains(keyword)] 
  return result

result = filter_keyword(df, "Verizon")
result

,Company Name,Stock Price
0,Verizon Communications Inc.,58.39


### condition

In [ ]:
def filter_range(data, range):
    result = data[data['Stock Price']>range] 
    return result

result = filter_range(df, 200) #>200 dollar
result

,Company Name,Stock Price
3,"Amazon.com, Inc.",3372.01
5,Tesla Inc,714.63


### sort

In [ ]:
df2 = df.sort_values(['Stock Price'], ascending=[0])
df2

,Company Name,Stock Price
3,"Amazon.com, Inc.",3372.01
5,Tesla Inc,714.63
1,"QUALCOMM, Inc.",135.25
2,Apple Inc,134.94
4,T-Mobile Us Inc,131.99
0,Verizon Communications Inc.,58.39
